In [ ]:
# Africa -> 0, 2103 total items
# Asia -> 1, 8852 total items
# Europe -> 2, 18117 total items
# North America -> 3, 14502 total items
# Oceania -> 4, 2296 total items
# South America -> 5, 4125 total items

In [2]:
import os
import torch
import torchvision
from torchvision import datasets, transforms
import sklearn
from tqdm import tqdm

In [21]:
def count_leaf_files(directory):
    try:
        children = os.listdir(directory)
        child_dirs = [os.path.join(directory, x) for x in children]
        total = 0
        total += sum([count_leaf_files(x) for x in child_dirs])
        return total
    except NotADirectoryError as e:
        return 1
        
class GeoGuessrDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.all_files = [x for x in os.listdir(self.img_dir) if 'jpg' in x]
        self.transform = transform

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self, idx):
        filename = self.all_files[idx]
        img = torchvision.io.read_image(os.path.join(self.img_dir, filename))
        #img = torchvision.transforms.functional.resize(img, [256, 256]) # may help ???
        if self.transform:
            img = self.transform(img)
            
        label = torch.tensor(int(filename[0]))
        if label >= 1:
            label = label - 1
        
        return img.float(), label.long()
            
        

    

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
transform = transforms.Compose([
    transforms.ConvertImageDtype(torch.float),
    transforms.Resize((256,256)),
    transforms.Normalize(mean=[0.5035, 0.5131, 0.4874], std = [0.2178, 0.2180, 0.2618])
])

In [24]:
dataset = GeoGuessrDataset('continents', transform=transform)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [25]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=8, shuffle=True, num_workers = 0, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=8, shuffle=True, num_workers = 0, pin_memory=True)

In [ ]:
mean = 0.
std = 0.
n_pixels = 0

for data, _ in tqdm(train_dataloader):
    batch_samples = data.size(0)
    channels = data.size(1)
    pixels_per_batch = batch_samples * data.size(2) * data.size(3)

    mean += data.sum(dim=[0, 2, 3])
    std += (data ** 2).sum(dim=[0, 2, 3])
    n_pixels += pixels_per_batch

mean /= n_pixels
std = (std / n_pixels - mean ** 2).sqrt()

print("Mean:", mean)
print("Std:", std)


In [27]:
class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = torch.nn.Conv2d(in_channels = in_channels, out_channels=out_channels, kernel_size = 3, padding=1)
        self.maxpool = torch.nn.MaxPool2d(2)
        self.activ = torch.nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.maxpool(x)
        x = self.activ(x)
        return x
        
class Model(torch.nn.Module):
    def __init__(self, input_shape=None):
        super().__init__()
        if input_shape == None:
            raise Exception("Input shape missing")

        
        self.block1 = ConvBlock(3, 16)
        self.block2 = ConvBlock(16, 32)
        self.block3 = ConvBlock(32, 64)
        self.flatten = torch.nn.Flatten(start_dim=1)
        with torch.no_grad():
            sample = torch.zeros(1, *input_shape)
            x = self.block1(sample)
            x = self.block2(x)
            x = self.block3(x)
            x = self.flatten(x)
            self.latent_dim = x.shape[1]
        
        self.dense1 = torch.nn.Linear(self.latent_dim, out_features=128)
        self.dense2 = torch.nn.Linear(in_features=128, out_features=64)
        self.dense3 = torch.nn.Linear(in_features=64, out_features=6)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        return x

In [28]:
sample = torch.randn(3, 3, 1536, 662).to(device)
resized_sample = torch.randn(3, 3, 256, 256).to(device)

In [29]:
model = Model(input_shape=(3, 256, 256)).to(device)

In [31]:
model.forward(resized_sample).shape

torch.Size([3, 6])

In [32]:
from sklearn.metrics import accuracy_score

In [39]:
def test(model):
    model.eval()
    y_total = []
    y_pred_total = []
    for X, y in tqdm(test_dataloader):
        X, y = X.to(device), y.to(device)
        outputs = model(X).float()
        y_pred = y_pred_total.append(torch.argmax(outputs, dim=1).cpu().numpy())
        y = y_total.append(y.cpu().numpy())
    return y_total, y_pred_total

In [41]:
y, y_pred = test(model)

100%|██████████████████████████████████████████████████████████████████████████████| 1249/1249 [01:10<00:00, 17.83it/s]


In [45]:
y = [list(x) for x in y]
y_pred = [list(x) for x in y_pred]

In [49]:
y = [x for x in [a for a in y]]

In [51]:
y_good = []
for a in y:
    y_good.extend(a)

In [53]:
y_pred_good = []
for a in y_pred:
    y_pred_good.extend(a)

In [55]:
accuracy_score(y_good, y_pred_good)

0.52563075690829

In [33]:
def train(model, epochs):

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()

    loss_hist = []
    acc_hist = []
    step_hist = []
    
    model.train()

    i = 0
    for epoch in range(epochs):
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            i += 1
            
            optimizer.zero_grad()
            outputs = model(X).float()
            loss = loss_fn(outputs, y)
            loss.backward()
            optimizer.step()
            
            if i % 200 == 0 and i > 200:
                y_pred = torch.argmax(outputs, dim=1).cpu().numpy()
                y = y.cpu().numpy()
                acc = accuracy_score(y, y_pred)
                
                print(f'Epoch loss: {loss.item() / len(y)}, acc: {acc}')
                # print(f'Epoch loss: {loss.item() / len(y)}')
                loss_hist.append(loss.item() / len(y))
                acc_hist.append(acc)
                step_hist.append(i)
            
    return loss_hist, acc_hist, step_hist


        

In [34]:
metrics = train(model, 3)

Epoch loss: 0.22432903945446014, acc: 0.25
Epoch loss: 0.19439999759197235, acc: 0.25
Epoch loss: 0.20679756999015808, acc: 0.25
Epoch loss: 0.20227986574172974, acc: 0.375
Epoch loss: 0.20169922709465027, acc: 0.25
Epoch loss: 0.13633376359939575, acc: 0.5
Epoch loss: 0.26577767729759216, acc: 0.125
Epoch loss: 0.22809475660324097, acc: 0.375
Epoch loss: 0.20120921730995178, acc: 0.125
Epoch loss: 0.28322476148605347, acc: 0.25
Epoch loss: 0.16155576705932617, acc: 0.5
Epoch loss: 0.14739269018173218, acc: 0.375
Epoch loss: 0.16490530967712402, acc: 0.5
Epoch loss: 0.20785440504550934, acc: 0.125
Epoch loss: 0.19996735453605652, acc: 0.375
Epoch loss: 0.1592976152896881, acc: 0.5
Epoch loss: 0.19464966654777527, acc: 0.375
Epoch loss: 0.16238349676132202, acc: 0.25
Epoch loss: 0.15046878159046173, acc: 0.375
Epoch loss: 0.14811421930789948, acc: 0.625
Epoch loss: 0.1004604622721672, acc: 0.625
Epoch loss: 0.14436259865760803, acc: 0.375
Epoch loss: 0.1948218196630478, acc: 0.25
Epoch 

In [ ]:
import matplotlib.pyplot as plt
import torchvision
import torch

img = dataset[1]
print(img)

# Dacă e un tuple (imagine, etichetă), extrage doar imaginea:
if isinstance(img, (tuple, list)):
    img = img[0]

# Dacă imaginea e un tensor PyTorch, convertește-l
if isinstance(img, torch.Tensor):
    img = img.permute(1, 2, 0)  # (C, H, W) -> (H, W, C)
    img = img.numpy()

# Dacă e nevoie, ajustează forma (opțional, dacă știi forma corectă)
# img = img.reshape((662, 1536, 3))  # doar dacă e o imagine plată

# Afișează imaginea
plt.imshow(img)
plt.axis('off')
plt.show()
